In [ ]:
from IPython.display import display, Javascript
import urllib.parse
import json

def load_code():
    # Method 1: Check URL parameters (works in JupyterLab 3+)
    try:
        from notebook import notebookapp
        servers = list(notebookapp.list_running_servers())
        if servers:
            url = servers[0]['url']
            if '?' in url:
                params = urllib.parse.parse_qs(url.split('?')[1])
                if 'fileurl' in params:
                    file_url = urllib.parse.unquote(params['fileurl'][0])
                    try:
                        import urllib.request
                        with urllib.request.urlopen(file_url) as response:
                            return response.read().decode('utf-8')
                    except Exception as e:
                        print(f"URL load failed: {e}")
    except:
        pass
    
    # Method 2: sessionStorage fallback
    js_code = """
    new Promise((resolve) => {
        let code = '';
        if (sessionStorage.getItem('lastGeneratedCodeURL')) {
            fetch(sessionStorage.getItem('lastGeneratedCodeURL'))
                .then(response => response.text())
                .then(text => {
                    code = text;
                    const cells = Jupyter.notebook.get_cells();
                    for (let i=0; i<cells.length; i++) {
                        if (cells[i].cell_type === 'code') {
                            cells[i].set_text(code);
                            resolve(code);
                            return;
                        }
                    }
                    resolve('');
                })
                .catch(() => resolve(''));
        } else {
            resolve('');
        }
    })
    """
    
    from IPython.display import Javascript
    display(Javascript(js_code))
    return None

# Load and inject code
import asyncio

async def inject_code():
    code = await load_code()
    if code:
        js_inject = f"""
        var code = {json.dumps(code)};
        var cells = Jupyter.notebook.get_cells();
        for (var i=0; i<cells.length; i++) {{
            if (cells[i].cell_type === 'code') {{
                cells[i].set_text(code);
                break;
            }}
        }}
        """
        display(Javascript(js_inject))

# Run the injection
await inject_code()

# Default imports
import control
import matplotlib.pyplot as plt
%matplotlib inline
print('Notebook ready for simulation')